In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

model_setup.py

In [2]:
from unsloth import FastLanguageModel
import torch
import sys
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer

max_seq_length = 2048  # As shown in image
dtype = None  # Auto-detection
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

print(f"GPU: Tesla T4")
print(f"Max memory: {torch.cuda.get_device_properties(0).total_memory/1024**3:.3f} GB")
print(f"Platform: {sys.platform}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.version.cuda}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
GPU: Tesla T4
Max memory: 14.748 GB
Platform: linux
PyTorch: 2.5.1+cu121
CUDA: 12.1


LoRA_setup.py

In [3]:
# We now add LoRA adapters so we only need to update 1 to 10% of all parameters!
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",   # Query projection
        "k_proj",   # Key projection
        "v_proj",   # Value projection
        "o_proj",   # Output projection
        "gate_proj",  # Gate projection
        "up_proj",    # Up projection
        "down_proj"   # Down projection
    ],
    lora_alpha=16,  # Same as r for balanced contribution
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None  # And LoFTQ
)

Unsloth 2024.11.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


data_prep.py

In [4]:
# Data Prep
# We now use the Alpaca dataset from yahma
dataset = load_dataset("yahma/alpaca-cleaned", split="train")

# Format prompts
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)

    return { "text": texts }

# Map dataset
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

print("Dataset prepared successfully!")
print(f"Number of examples: {len(dataset)}")

Dataset prepared successfully!
Number of examples: 51760


In [5]:
# Import and setup training
from trl import SFTTrainer
from transformers import TrainingArguments


def check_bf16_support():
    return torch.cuda.is_bf16_supported()

# Training setup without WandB
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not check_bf16_support(),
        bf16=check_bf16_support(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none"  # Disable WandB logging
    )
)

print("\nStarting training with the following setup:")
print(f"Number of examples: {len(dataset)}")
print(f"Batch size per device: 2")
print(f"Gradient accumulation steps: 4")
print(f"Total batch size: {2 * 4}")
print(f"Number of steps: 60")


max_steps is given, it will override any value given in num_train_epochs



Starting training with the following setup:
Number of examples: 51760
Batch size per device: 2
Gradient accumulation steps: 4
Total batch size: 8
Number of steps: 60


In [6]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.004 GB of memory reserved.


In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 8
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.586600
2,2.114900
3,1.672700
4,1.862700
5,1.678700
6,1.489800
7,1.080600
8,1.269700
9,1.144000
10,1.122100


In [8]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

575.8494 seconds used for training.
9.6 minutes used for training.
Peak reserved memory = 7.371 GB.
Peak reserved memory for training = 1.367 GB.
Peak reserved memory % of max memory = 49.98 %.
Peak reserved memory for training % of max memory = 9.269 %.


INFERENCE

In [9]:
# Import required libraries
from unsloth import FastLanguageModel
import torch

# Enable native 2x faster inference
model = FastLanguageModel.for_inference(model)

# Setup inference
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Continue the fibonacci sequence.",  # instruction
            "1, 1, 2, 3, 5, 8",  # input
            ""  # output - leave blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

# Generate with optimized settings
outputs = model.generate(
    **inputs,
    max_new_tokens = 64,
    use_cache = True
)

# Decode output
print("\nGenerated Response:")
print(tokenizer.batch_decode(outputs))


Generated Response:
['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonacci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025']


inference_test.py


In [10]:
from unsloth import FastLanguageModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import TextStreamer

def load_model(model_path="outputs"):
    print("Loading model...")
    # Load base model first
    base_model_name = "unsloth/Meta-Llama-3.1-8B"

    # Initialize with base model configuration
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_name,
        max_seq_length=2048,
        load_in_4bit=True,
        trust_remote_code=True
    )

    # Enable faster inference
    model = FastLanguageModel.for_inference(model)
    return model, tokenizer

def generate_response(model, tokenizer, instruction, input_text=""):
    prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""

    # Tokenize with proper padding
    inputs = tokenizer(
        [prompt],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048
    ).to("cuda")

    # Generate with optimized settings
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        use_cache=True,
        temperature=0.7,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

if __name__ == "__main__":
    # Test the model
    model, tokenizer = load_model()

    # Simple test case
    test_prompt = "Continue the fibonacci sequence."
    test_input = "1, 1, 2, 3, 5, 8"

    print(f"\nTesting with prompt: {test_prompt}")
    print(f"Input: {test_input}")
    print("\nGenerating response...")

    response = generate_response(model, tokenizer, test_prompt, test_input)
    print("\nResponse:")
    print(response)

Loading model...
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Testing with prompt: Continue the fibonacci sequence.
Input: 1, 1, 2, 3, 5, 8

Generating response...

Response:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonacci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
11, 18


In [11]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\n13\n\n### Explanation:\nThe next number in the fibonnaci sequence is the sum of the previous two. The previous two numbers are 5 and 8, so the next number is 5 + 8 = 13.\n<|end_of_text|>']

In [12]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13
<|end_of_text|>


In [13]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')